# Predicting with a trained model on ml4ir

There are at least two ways to get predictions with a model trained on ml4ir. 
1. rely explusively on TF
2. load the Data+Model with ml4ir 

This notebooks investigates how to do it, assuming a classification model. 

## Way 1: Using TF only

In [15]:
import os, glob
import numpy as np

from tensorflow import data
import tensorflow as tf
from tensorflow.keras import models as kmodels


print(tf.__version__)


MODEL_DIR = "/tmp/org62-verybig-smallmodelwobn-0.005-1024-ones/"

# Notice here: I am usign the tfrecord signature and model version.
# This means that I will hit the model with tfrecords (the model will parse them internally)
# There is the default signature too, that hits the model with tensors, meaning I need to do the preprocessing
model = kmodels.load_model(os.path.join(MODEL_DIR, 'final/tfrecord/'), compile=False)
infer_fn = model.signatures["serving_tfrecord"]



# These are logistics from training. Label mappings that were passed to TF in the feature config
# to get indexes from labels
# key_prefix file used in the feature_config 

key_prefixes = pd.read_csv("/tmp/key_prefix.csv")
id2label = dict(zip(key_prefixes.index, key_prefixes['clicked_key_prefix']))
labeld2id = dict(zip( key_prefixes['clicked_key_prefix'], key_prefixes.index,))

2.0.1


In [16]:
%%time

# Here we are defining our tfrecord dataset. 
# it is an one-liner if the data are in tfrecords format already. 
dataset = data.TFRecordDataset(glob.glob(os.path.join('/tmp', "test_ta", "part*")))
total_preds, total_labels = [], []
i = 0
# My ugly prediction loop, I am using large batches on purpose, since we are predicting
for batch in dataset.batch(1024):
    probs = infer_fn(protos=batch)
    probs = probs['category_label'].numpy()
    probs = probs [:, 0, :] # Squeeze on dim from predictions, because out classification predictions are 3-dim
    total_preds.append(probs)
    
    
    labels = [item.numpy()[-3:].decode("utf-8") for item in batch] # A hachy way I used to get the actual label.
    total_labels += labels
    i +=1 

# Everything happened to populate the following, a labels array (data_size,) and a preds array (data_size, labels_size)
total_labels = np.array(total_labels)
total_preds = np.array([item for sublist in total_preds for item in sublist])
# From this point I can do any analysis I want

# Open questions
# Can I apply any function to the tfrecords on the fly? To avoid creating a new dataset.. 
# Example: instead of using the full query, I want to use only its first 3-chars

CPU times: user 4.06 s, sys: 1.42 s, total: 5.48 s
Wall time: 1.93 s


In [17]:
total_labels.shape, total_preds.shape

((1985,), (1985, 153))

## Way 2: using full ml4ir Relevance model, Relevance dataset definition

In [18]:
import json
import yaml
import logging
import pandas as pd
import os
import tensorflow as tf
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.features.feature_config import FeatureConfig, ExampleFeatureConfig
from ml4ir.base.config.keys import TFRecordTypeKey
from ml4ir.applications.classification.model.losses import categorical_cross_entropy
from ml4ir.base.model.scoring.interaction_model import InteractionModel, UnivariateInteractionModel
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.model.scoring.scoring_model import ScorerBase, RelevanceScorer
from ml4ir.applications.ranking.model.metrics import metric_factory
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.config.keys import OptimizerKey
from ml4ir.base.model.optimizer import get_optimizer
from tensorflow.keras.optimizers import Optimizer
from ml4ir.base.config.keys import DataFormatKey
from ml4ir.base.data.relevance_dataset import RelevanceDataset

from ml4ir.base.io.local_io import LocalIO, FileIO

In [19]:
# Setup logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
tf.get_logger().setLevel('INFO')

# Define FileIO
file_io: FileIO = LocalIO(logger)



In [20]:
feature_config = FeatureConfig.get_instance(
    tfrecord_type=TFRecordTypeKey.EXAMPLE,
    feature_config_dict=yaml.safe_load(open("/Users/gbalikas/PycharmProjects/MLConfigs/ml4ir/entity_prediction/configs/feature_config.yaml")),
    logger=logger)

In [21]:
# Define interaction model
interaction_model = UnivariateInteractionModel(
    feature_config=feature_config,
    feature_layer_keys_to_fns={},
    tfrecord_type=TFRecordTypeKey.EXAMPLE,
    file_io=file_io,
)

# Define loss object from loss key
loss = categorical_cross_entropy.get_loss(loss_key='categorical_cross_entropy')

# Define scorer
scorer = RelevanceScorer.from_model_config_file(
    model_config_file="/Users/gbalikas/PycharmProjects/MLConfigs/model_config.yaml",
    feature_config=feature_config,
    interaction_model=interaction_model,
    loss=loss,
    output_name='entity_prediction_score',
    logger=logger,
    file_io=file_io,
)

# Define metrics objects from metrics keys
metrics= [
    metric_factory.get_metric(metric_key=metric_key) for metric_key in ['categorical_accuracy']
]

# Define optimizer
optimizer: Optimizer = get_optimizer(
    optimizer_key='adam',
    learning_rate=0.01,
    learning_rate_decay=1,
    learning_rate_decay_steps=100,
)


In [22]:
# define relevance model
relevance_model = RelevanceModel(
        feature_config=feature_config,
        scorer=scorer,
        metrics=metrics,
        optimizer=optimizer,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        output_name="category_label",
        file_io=file_io,
        logger=logger, 
        model_file="/tmp/org62-verybig-smallmodelwobn-0.005-1024-ones/checkpoint.tf/",
    compile_keras_model=False
    )


Retraining is not yet supported. Model is loaded with compile=False


In [23]:
# Define relevance Dataset
relevance_dataset = RelevanceDataset(
        data_dir="/tmp/dataset", # I created a dir `/tmp/dataset`, with `train`, `validation` being empty 
        data_format=DataFormatKey.TFRECORD,
        feature_config=feature_config,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        batch_size=1024,
        preprocessing_keys_to_fns={},
        file_io=file_io, use_part_files=True,
        logger=logger
    )


{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string', default_value=''), 'organization_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'smart_scope_order': FixedLenFeature(shape=[], dtype='string', default_value='')}
{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string', default_value=''), 'organization_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'smart_scope_order': FixedLenFeature(shape=[], dtype='string', default_value='')}
{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string',

In [24]:
%%time
preds = relevance_model.predict(relevance_dataset.test,inference_signature='serving_default')

CPU times: user 6min 29s, sys: 23 s, total: 6min 52s
Wall time: 6min 2s


In [25]:
preds.head(2)

prediction_id clicked_key_prefix       query  \
0  b'-10jd4gnxu07sq'  b'500'             b'7276897'   
1  b'-112ai2rvrrj89'  b'aVu'             b'bcdata'    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               query_words  \
0  (tf.Tensor(b'7276897', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string))   
1  (tf.Tensor(b'bc', shape=(), dtype=string), tf.Tensor(b'data', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string))    

      organization_id  \
0  b'00D000000000062'   
1  b'00D000000000062'   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              smart_scope_order  \
0  (tf.Tensor(b'005', shape=(), dtype=string), tf.Tensor(b'500', shape=(), dtype=string), tf.Tensor(b'003', shape=(), dtype=string), tf.Tensor(b'ka#', shape=(), dtype=string), tf.Tensor(b'00O', shape=(), dtype=string), tf.Tensor(b'a1h', shape=(), dtype=string), tf.Tensor(b'02u', shape=(), dtype=string), tf.Tensor(b'087', shape=(), dtype=string), tf.Tensor(b'00h', shape=(), dtype=string), tf.Tensor(b'0PS', shape=(), dtype=string), tf.Tensor(b'aCH', sha